In [13]:
!pip install pyDOE

In [1]:
import tensorflow as tf
import datetime, os
#hide tf logs 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'} 
#0 (default) shows all, 1 to filter out INFO logs, 2 to additionally filter out WARNING logs, and 3 to additionally filter out ERROR logs
import scipy.optimize
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import seaborn as sns 
import codecs, json

# generates same random numbers each time
np.random.seed(1234)
tf.random.set_seed(1234)

print("TensorFlow version: {}".format(tf.__version__))

TensorFlow version: 2.8.0


# *Data Prep*

Training and Testing data is prepared from the solution file

In [2]:
from scipy.stats import multivariate_normal
#getting collocation points
x = np.linspace(-1, 1, 128)                     # 256 points between -1 and 1 [256x1]
y = np.linspace(-1, 1, 128)
t = np.linspace(0, 0.2, 100)                    # 100 time points between 0 and 1 [100x1]
X, Y, T = np.meshgrid(x, y ,t) 
usol=np.zeros((128, 128 ,100))
pos = np.dstack((X[:,:,0], Y[:,:,0]))
rv = multivariate_normal(mean=[0.0 , 0.0], cov=[0.05 , 0.05])
usol[:][:,:,0] = rv.pdf(pos)

#collocation points for every position and every time



# *Test Data*

We prepare the test data to compare against the solution produced by the PINN.

In [3]:
''' X_u_test = [X[i],T[i]] [25600,2] for interpolation'''
X_u_test = np.hstack((X.flatten()[:,None], Y.flatten()[:,None], T.flatten()[:,None]))

# Domain bounds
lb = X_u_test[0]  # [-1. 0.]
ub = X_u_test[-1] # [1.  0.99]

'''
   Fortran Style ('F') flatten,stacked column wise!
   u = [c1 
        c2
        .
        .
        cn]

   u =  [25600x1] 
'''
u = usol.flatten('F')[:,None] 

# *Training Data*

The boundary conditions serve as the test data for the PINN and the collocation points are generated using **Latin Hypercube Sampling**

In [4]:
def trainingdata(N_u,N_f):

    '''Boundary Conditions'''

    #Initial Condition -1 =< x =<1 and t = 0  
    leftedge_x = np.hstack((X[0,:][:,None][:,:,0], X[0,:][:,None][:,:,0], T[0,:][:,None][:,:,0])) #L1
    leftedge_u = usol[:,0][:,None]

    #Boundary Condition x = -1 and 0 =< t =<1
    bottomedge_x = np.hstack((X[:,0][:,None][0].T, X[0,:][:,None][0].T, T[:,0][:,None][0].T)) #L2
    bottomedge_u = usol[-1,:][:,None]

    #Boundary Condition x = 1 and 0 =< t =<1
    topedge_x = np.hstack((X[:,-1][:,None][0].T, X[0,:][:,None][0].T, T[:,0][:,None][0].T)) #L3
    topedge_u = usol[0,:][:,None]

    all_X_u_train = np.vstack([leftedge_x, bottomedge_x, topedge_x]) # X_u_train [456,2] (456 = 256(L1)+100(L2)+100(L3))
    all_u_train = np.vstack([leftedge_u, bottomedge_u, topedge_u])   #corresponding u [456x1]

    #choose random N_u points for training
    idx = np.random.choice(all_X_u_train.shape[0], N_u, replace=False) 

    X_u_train = all_X_u_train[idx, :] #choose indices from  set 'idx' (x,t)
    u_train = all_u_train[idx,:]      #choose corresponding u

    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    X_f_train = lb + (ub-lb)*lhs(3,N_f) 
    X_f_train = np.vstack((X_f_train, X_u_train)) # append training points to collocation points 

    return X_f_train, X_u_train, u_train 


# **PINN**

Generate a **PINN** of L hidden layers, each with n neurons. 

Initialization: ***Xavier***

Activation: *tanh (x)*

In [5]:
class Sequentialmodel(tf.Module): 
    def __init__(self, layers, name=None):
        self.itera = 0
        self.W = []  #Weights and biases
        self.parameters = 0 #total number of parameters
        
        for i in range(len(layers)-1):
            
            input_dim = layers[i]
            output_dim = layers[i+1]
            
            #Xavier standard deviation 
            std_dv = np.sqrt((2.0/(input_dim + output_dim)))

            #weights = normal distribution * Xavier standard deviation + 0
            w = tf.random.normal([input_dim, output_dim], dtype = 'float64') * std_dv
                       
            w = tf.Variable(w, trainable=True, name = 'w' + str(i+1))

            b = tf.Variable(tf.cast(tf.zeros([output_dim]), dtype = 'float64'), trainable = True, name = 'b' + str(i+1))
                    
            self.W.append(w)
            self.W.append(b)
            
            self.parameters +=  input_dim * output_dim + output_dim
    
    def evaluate(self,x):
        
        x = (x-lb)/(ub-lb)
        
        a = x
        
        for i in range(len(layers)-2):
            
            z = tf.add(tf.matmul(a, self.W[2*i]), self.W[2*i+1])
            a = tf.nn.tanh(z)
            
        a = tf.add(tf.matmul(a, self.W[-2]), self.W[-1]) # For regression, no activation to last layer
        return a
    
    def get_weights(self):

        parameters_1d = []  # [.... W_i,b_i.....  ] 1d array
        
        for i in range (len(layers)-1):
            
            w_1d = tf.reshape(self.W[2*i],[-1])   #flatten weights 
            b_1d = tf.reshape(self.W[2*i+1],[-1]) #flatten biases
            
            parameters_1d = tf.concat([parameters_1d, w_1d], 0) #concat weights 
            parameters_1d = tf.concat([parameters_1d, b_1d], 0) #concat biases
        
        return parameters_1d
        
    def set_weights(self,parameters):
                
        for i in range (len(layers)-1):

            shape_w = tf.shape(self.W[2*i]).numpy() # shape of the weight tensor
            size_w = tf.size(self.W[2*i]).numpy() #size of the weight tensor 
            
            shape_b = tf.shape(self.W[2*i+1]).numpy() # shape of the bias tensor
            size_b = tf.size(self.W[2*i+1]).numpy() #size of the bias tensor 
                        
            pick_w = parameters[0:size_w] #pick the weights 
            self.W[2*i].assign(tf.reshape(pick_w,shape_w)) # assign  
            parameters = np.delete(parameters,np.arange(size_w),0) #delete 
            
            pick_b = parameters[0:size_b] #pick the biases 
            self.W[2*i+1].assign(tf.reshape(pick_b,shape_b)) # assign 
            parameters = np.delete(parameters,np.arange(size_b),0) #delete 

            
    def loss_BC(self,x,y):

        loss_u = tf.reduce_mean(tf.square(y-self.evaluate(x)))
        return loss_u

    def loss_PDE(self, x_to_train_f):
    
        g = tf.Variable(x_to_train_f, dtype = 'float64', trainable = False)
    
        cost=10 
        sigma2=0.25

        x_f = g[:,0:1]
        y_f = g[:,1:2]
        t_f = g[:,2:3]

        with tf.GradientTape(persistent=True) as tape:

            tape.watch(x_f)
            tape.watch(y_f)
            tape.watch(t_f)

            g = tf.stack([x_f[:,0], y_f[:,0], t_f[:,0]], axis=1)   

            z = self.evaluate(g)
            p_x = tape.gradient(z,x_f)
            p_y = tape.gradient(z,y_f)

        p_t = tape.gradient(z,t_f)    
        p_xx = tape.gradient(p_x, x_f)
        p_yy = tape.gradient(p_y, y_f)
        p_xy = tape.gradient(p_x, y_f)
        p_yx = tape.gradient(p_y, x_f)

        del tape

        p=self.evaluate(g)

        f = p_t - sigma2/2*p_xx - sigma2/2*p_yy

        loss_f = tf.reduce_mean(tf.square(f))

        return loss_f
    
    def loss(self,x,y,g):

        loss_u = self.loss_BC(x,y)
        loss_f = self.loss_PDE(g)

        loss = loss_u + loss_f

        return loss, loss_u, loss_f
    
    def optimizerfunc(self,parameters):
        
        self.set_weights(parameters)
       
        with tf.GradientTape() as tape:
            tape.watch(self.trainable_variables)
            
            loss_val, loss_u, loss_f = self.loss(X_u_train, u_train, X_f_train)
            
        grads = tape.gradient(loss_val,self.trainable_variables)
                
        del tape
        
        grads_1d = [ ] #flatten grads 
        
        for i in range (len(layers)-1):

            grads_w_1d = tf.reshape(grads[2*i],[-1]) #flatten weights 
            grads_b_1d = tf.reshape(grads[2*i+1],[-1]) #flatten biases

            grads_1d = tf.concat([grads_1d, grads_w_1d], 0) #concat grad_weights 
            grads_1d = tf.concat([grads_1d, grads_b_1d], 0) #concat grad_biases

        return loss_val.numpy(), grads_1d.numpy()
    
    def optimizer_callback(self,parameters):
               
        loss_value, loss_u, loss_f = self.loss(X_u_train, u_train, X_f_train)
        
        u_pred = self.evaluate(X_u_test)
        error_vec = np.linalg.norm((u-u_pred),2)/np.linalg.norm(u,2)
        
        tf.print(self.itera, loss_value, loss_u, loss_f, error_vec)
        self.itera +=1

# *Solution Plot*

In [6]:
def solutionplot(u_pred,X_u_train,u_train):
    
    fig, ax = plt.subplots()
    ax.axis('off')

    gs0 = gridspec.GridSpec(2, 3)
    gs0.update(top=1, bottom=0, left=0.1, right=2, wspace=0.3, hspace =0.4)
    ax = plt.subplot(gs0[0, :])

    h = ax.imshow(u_pred, interpolation='nearest', cmap='rainbow', 
                extent=[T.min(), T.max(), X.min(), X.max()], 
                origin='lower', aspect='auto')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(h, cax=cax)
    
    ax.plot(X_u_train[:,1], X_u_train[:,0], 'kx', label = 'Data (%d points)' % (u_train.shape[0]), markersize = 4, clip_on = False)

    line = np.linspace(x.min(), x.max(), 2)[:,None]
    #ax.plot(t[25]*np.ones((2,1)), line, 'w-', linewidth = 1)
    #ax.plot(t[50]*np.ones((2,1)), line, 'w-', linewidth = 1)
    #ax.plot(t[75]*np.ones((2,1)), line, 'w-', linewidth = 1)    

    ax.set_xlabel('$t$')
    ax.set_ylabel('$x$')
    ax.legend(frameon=False, loc = 'best')
    ax.set_title('$u(x,t)$', fontsize = 10)
    
    ''' 
    Slices of the solution at points t = 0.25, t = 0.50 and t = 0.75
    '''
    
    ####### Row 1: u(t,x) slices ##################
    #gs1 = gridspec.GridSpec(1, 3)
    #gs1.update(top=0.3, bottom=-0.1, left=0.1, right=2, wspace=0.5)

    ax = plt.subplot(gs0[1, 0])
    #ax.plot(x,usol.T[0,:], 'b-', linewidth = 2, label = 'Exact')       
    ax.plot(x,u_pred.T[0,:], 'r', linewidth = 2, label = 'Prediction')
    ax.set_xlabel('$x$')
    ax.set_ylabel('$u(x,t)$')    
    ax.set_title('$t = 0.s$', fontsize = 10)
    #ax.axis('square')
    ax.set_xlim([-1.1,1.1])
    ax.set_ylim([-0.1,9])

    ax = plt.subplot(gs0[1, 1])
    #ax.plot(x,usol.T[50,:], 'b-', linewidth = 2, label = 'Exact')       
    ax.plot(x,u_pred.T[500,:], 'r', linewidth = 2, label = 'Prediction')
    ax.set_xlabel('$x$')
    ax.set_ylabel('$u(x,t)$')
    #ax.axis('square')
    ax.set_xlim([-1.1,1.1])
    ax.set_ylim([-0.1,9])
    ax.set_title('$t = 0.1s$', fontsize = 10)
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.35), ncol=5, frameon=False)

    ax = plt.subplot(gs0[1, 2])
    #ax.plot(x,usol.T[75,:], 'b-', linewidth = 2, label = 'Exact')       
    ax.plot(x,u_pred.T[750,:], 'r', linewidth = 2, label = 'Prediction')
    ax.set_xlabel('$x$')
    ax.set_ylabel('$u(x,t)$')
    #ax.axis('square')
    ax.set_xlim([-1.1,1.1])
    ax.set_ylim([-0.1,9])    
    ax.set_title('$t = 0.15s$', fontsize = 10)
    
    #plt.tight_layout()
    plt.savefig('Ornstein-Uhlenbeck.png',dpi = 500)   

# *Model Training and Testing*

A function '**model**' is defined to generate a NN as per the input set of hyperparameters, which is then trained and tested. The L2 Norm of the solution error is returned as a comparison metric

In [26]:
N_u = 300 #Total number of data points for 'u'
N_f = 30000 #Total number of collocation points 

# Training data
X_f_train, X_u_train, u_train = trainingdata(N_u, N_f)

layers = np.array([3,20,20,20,20,20,20,20,20,1]) #8 hidden layers

PINN = Sequentialmodel(layers)

init_params = PINN.get_weights().numpy()

start_time = time.time() 

# train the model with Scipy L-BFGS optimizer
results = scipy.optimize.minimize(fun = PINN.optimizerfunc, 
                                  x0 = init_params, 
                                  args=(), 
                                  method='L-BFGS-B', 
                                  jac= True,        # If jac is True, fun is assumed to return the gradient along with the objective function
                                  callback = PINN.optimizer_callback, 
                                  options = {'disp': None,
                                            'maxcor': 200, 
                                            'ftol': 1 * np.finfo(float).eps,  #The iteration stops when (f^k - f^{k+1})/max{|f^k|,|f^{k+1}|,1} <= ftol
                                            'gtol': 5e-8, 
                                            'maxfun':  50000, 
                                            'maxiter': 1000,
                                            'iprint': -1,   #print update every 50 iterations
                                            'maxls': 50})

elapsed = time.time() - start_time                
print('Training time: %.2f' % (elapsed))

print(results)

PINN.set_weights(results.x)

''' Model Accuracy ''' 
u_pred = PINN.evaluate(X_u_test)

error_vec = np.linalg.norm((u-u_pred),2)/np.linalg.norm(u,2)        # Relative L2 Norm of the error (Vector)
print('Test Error: %.5f'  % (error_vec))

#u_pred = np.reshape(u_pred,(256,1000),order='F')                        # Fortran Style ,stacked column wise!

''' Solution Plot '''
#solutionplot(u_pred,X_u_train,u_train)

0 0.012534742170616977 0.00154855434373108 0.010986187826885896 1.311236329887067
1 0.01116461388930518 0.0014454664014550043 0.0097191474878501761 1.3615971690664503
2 0.0072524945445896164 0.0014699346480054251 0.0057825598965841911 1.487353409760186
3 0.0041980671171782721 0.00083444192531202926 0.0033636251918662424 1.258846154290494
4 0.0022082662136348415 0.00073613552871338911 0.0014721306849214523 1.1305504935985045
5 0.0015050279695727419 0.000729846482230578 0.00077518148734216392 1.0683364559712631
6 0.00099795248909520583 0.00056046761971835133 0.00043748486937685444 1.051995372761837
7 0.00084272611218794938 0.00046596630706242215 0.00037675980512552723 1.0457505171739074
8 0.000603806158407285 0.00025956433011024757 0.00034424182829703742 1.019426363909217
9 0.00045637975515174948 0.00011596770610630592 0.00034041204904544355 1.0091042210898555
10 0.00028368036188122135 3.1658038206609853e-05 0.00025202232367461149 1.0087642479938546
11 0.00020664590685793159 1.9462740023

92 1.2015554010505809e-07 1.1504798516136288e-08 1.086507415889218e-07 0.999662209811048
93 1.1877235053306029e-07 1.1819723142732143e-08 1.0695262739032815e-07 0.9996604629084314
94 1.1426981712004643e-07 1.1418155205592413e-08 1.0285166191445402e-07 0.9996572812141635
95 1.0753402521771911e-07 8.48126000708442e-09 9.9052765210634689e-08 0.9996431097162253
96 1.0108775461478437e-07 3.6407039798212988e-09 9.7447050634963064e-08 0.9996150476234156
97 9.6543909181779168e-08 2.445729093626133e-09 9.4098180088153039e-08 0.9996089987725748
98 9.1930527656847524e-08 1.9188412753375455e-09 9.0011686381509977e-08 0.9996056945871877
99 8.9747525937664759e-08 2.1611838612737916e-09 8.7586342076390974e-08 0.9995974048786318
100 8.8200363400697342e-08 2.9314795750108309e-09 8.52688838256865e-08 0.9996126662863966
101 8.6804094297795024e-08 3.0104623778151383e-09 8.3793631919979888e-08 0.9995955881482078
102 8.6150551546633137e-08 2.6534262216461706e-09 8.3497125324986967e-08 0.9995909437250464
103

182 1.4915955286434337e-08 1.01862650991755e-09 1.3897328776516787e-08 0.9998187994871417
183 1.4755468661033795e-08 1.1390843493510709e-09 1.3616384311682724e-08 0.9998232220125136
184 1.4467969733496523e-08 1.2959742263746304e-09 1.3171995507121893e-08 0.9998297101283509
185 1.4372817913421482e-08 1.3597827048650151e-09 1.3013035208556467e-08 0.9998305097235403
186 1.408085212039635e-08 1.3570724193143003e-09 1.2723779701082049e-08 0.9998317963436247
187 1.3892288480130411e-08 1.2407836192334502e-09 1.2651504860896961e-08 0.9998284830746892
188 1.3782445024745355e-08 1.1389636659310342e-09 1.2643481358814321e-08 0.9998231884086539
189 1.3704768317893086e-08 1.0511771897369208e-09 1.2653591128156165e-08 0.99981715667
190 1.3639877376958847e-08 9.9624834876409418e-10 1.2643629028194752e-08 0.9998120977218281
191 1.3588898141243861e-08 9.9037341617028187e-10 1.259852472507358e-08 0.9998100449439395
192 1.3571139052418782e-08 9.8454465661465175e-10 1.258659439580413e-08 0.999808500375619

272 5.6872683228536754e-09 4.3116991963515646e-10 5.2560984032185192e-09 0.9998083744217312
273 5.6777391423931106e-09 4.2430475415504468e-10 5.253434388238066e-09 0.9998073234909569
274 5.667139272214153e-09 4.177563798362195e-10 5.2493828923779335e-09 0.9998063639865253
275 5.6477568327653145e-09 4.1275792770172691e-10 5.2349989050635876e-09 0.999804756413742
276 5.6081751195051058e-09 4.0328865551287816e-10 5.2048864639922277e-09 0.9998023412099937
277 5.53661348067079e-09 3.9160846520394644e-10 5.1450050154668438e-09 0.9997995331748608
278 5.4670956031399504e-09 3.6898687141126892e-10 5.0981087317286816e-09 0.9997968951491656
279 5.2743695869009748e-09 3.244460915874646e-10 4.94992349531351e-09 0.999796937322253
280 5.182438661724753e-09 2.9652364948821607e-10 4.8859150122365366e-09 0.9997986437743098
281 5.0566778164565938e-09 2.910402058710219e-10 4.7656376105855715e-09 0.9997999628787464
282 5.0180567689591613e-09 2.948956037731571e-10 4.7231611651860041e-09 0.9997964542392979
2

363 2.5579557890296644e-09 1.6155850026918351e-10 2.3963972887604809e-09 0.9998553462191282
364 2.5574057469119586e-09 1.6107481491070636e-10 2.3963309320012522e-09 0.9998551903947355
365 2.5566780212939869e-09 1.6074232747539829e-10 2.3959356938185887e-09 0.9998548912996034
366 2.555304606732659e-09 1.6006450070578077e-10 2.3952401060268782e-09 0.9998545349661957
367 2.5527111271794343e-09 1.5898751717231981e-10 2.3937236100071146e-09 0.9998539714995263
368 2.5473499985375669e-09 1.5692518700495457e-10 2.3904248115326125e-09 0.999853344185113
369 2.5361109519232356e-09 1.5393028562345984e-10 2.3821806662997759e-09 0.9998525821708105
370 2.5127234162140695e-09 1.4962573035116261e-10 2.3630976858629067e-09 0.9998523886209156
371 2.4820255245110434e-09 1.5061083941294915e-10 2.3314146850980941e-09 0.9998511186866598
372 2.4320175428797683e-09 1.4177282341823599e-10 2.2902447194615324e-09 0.9998542498934817
373 2.3935978011783434e-09 1.650258547537325e-10 2.2285719464246107e-09 0.99985197

' Solution Plot '

In [27]:
u_pred = np.reshape(u_pred,(128, 128, 100), order='F')   

In [28]:
def plot_3d_ic(time):
    fig = plt.figure(figsize=(10, 10))
    ax = plt.axes(projection='3d')
    ax.plot_surface(X[:,:,0], Y[:,:,0], usol[:,:,time])

def plot_3d_pred(time):
    fig = plt.figure(figsize=(10, 10))
    ax = plt.axes(projection='3d')
    ax.plot_surface(X[:,:,0], Y[:,:,0], u_pred[:,:,time])
    
    
def plot_2d(time):
    plt.plot(x, u_pred_2[0][:,time])
    #plt.ylim(0, 0.2)
    #plt.xlim(-1, 1)

In [29]:
import ipywidgets

        
ipywidgets.interact(plot_3d_ic, time=(0, 99, 2))
ipywidgets.interact(plot_3d_pred, time=(0, 99, 2))

interactive(children=(IntSlider(value=48, description='time', max=99, step=2), Output()), _dom_classes=('widge…

interactive(children=(IntSlider(value=48, description='time', max=99, step=2), Output()), _dom_classes=('widge…

<function __main__.plot_3d_pred(time)>